<a href="https://colab.research.google.com/github/DylanCTY/TextAnalyticsProject_Group5/blob/main/Gender_Recognisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (1,717 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121920 files and directories currently installed.)
P

In [ ]:
!pip install PyAudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyAudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63858 sha256=e5ee1eb7decee771a3c700c7f09696d0c24909f403319802f30174fbfe6f55f4
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built PyAudio


Build Gender Prediction Model using SVM Classification Based on Mean, Min, Max Frequency

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
import wave
import os
import platform



def import_and_clean():
    df = pd.read_csv('training_data.csv', header=0)

    df = df[['meanfun' , 'minfun', 'maxfun','label']]
    df['label'] = df['label'].map({'female': 0, 'male': 1}).astype(int)
    return df

def parameter_tuning_svm(input_df):
    x = input_df['meanfun' , 'minfun', 'maxfun'].values
    y = input_df['label'].values
   #svc = SVC(kernel='linear')

    #segmenting data set and cross validation
    training, testing, training_result, testing_result = train_test_split(x, y, test_size=0.4, random_state=1)
    # scores = cross_val_score(svc, training, training_result, cv=10, scoring='accuracy')
    # print scores.mean()

    #Tuning C value
    c_vals = list(range(1,30))
    accuracy_vals = []
    for val in c_vals:
        svc = SVC(kernel='linear', C=val)
        scores = cross_val_score(svc, training, training_result, cv=10, scoring='accuracy')
        accuracy_vals.append(scores.mean())

    # plt.plot(c_vals, accuracy_vals)
    # plt.xticks(np.arange(0,30,2))
    # plt.xlabel('C values')
    # plt.ylabel('Mean Accuracies')
    # plt.show()

    optimal_cval = c_vals[accuracy_vals.index(max(accuracy_vals))]
    print(optimal_cval)

    #gamma value tuning
    gamma_vals = [0.00001,0.0001,0.001,0.01,0.1]
    accuracy_vals = []
    for g in gamma_vals:
        svc = SVC(kernel='linear', C=optimal_cval, gamma=g)
        scores = cross_val_score(svc, training, training_result, cv=10, scoring='accuracy')
        accuracy_vals.append(scores.mean())
    #
    # plt.plot(gamma_vals, accuracy_vals)
    # plt.xlabel('Gamma Values')
    # plt.ylabel('Mean Accuracies')
    # plt.show()

    optimal_gamma = gamma_vals[accuracy_vals.index(max(accuracy_vals))]
    #print optimal_gamma

    svc = SVC(kernel='linear', C=optimal_cval, gamma=optimal_gamma)
    svc.fit(training, training_result)
    testing_predict = svc.predict(testing)
    #print metrics.accuracy_score(testing_predict, testing_result)

    svc = SVC(kernel='linear', C=optimal_cval, gamma=optimal_gamma)
    svc.fit(x,y)
    return svc



In [ ]:
#check training data
x = df[['meanfun' , 'minfun', 'maxfun']].values
x

array([[0.08427911, 0.01570167, 0.27586207],
       [0.10793655, 0.01582591, 0.25      ],
       [0.09870626, 0.01565558, 0.27118644],
       ...,
       [0.20991768, 0.03950617, 0.27586207],
       [0.172375  , 0.03448276, 0.25      ],
       [0.18560693, 0.06225681, 0.27118644]])

In [ ]:
#import package to extract sound
#still not working on my pc so i just tried using the sample output.txt, will need to check deeper
#should have produced the output.txt from the test_audio.wav
if __name__ == '__main__':
    if platform.system() == 'Linux':
        os.system('"Praat.exe" --run extract_freq_info.praat')
    elif platform.system() == 'Windows':
        os.system('"./Praat" --run extract_freq_info.praat')
    else:
        os.system('"Praat.app/Contents/MacOS/Praat" --run extract_freq_info.praat')

In [ ]:
#produce the mean, min, and max frequency
file = open('output.txt','r')
values = file.readline()
values = values.split(', ')
values = np.array(values, dtype=np.float64)
values

array([129.87966696, 103.80061643, 144.31090385])

In [ ]:
#complete code to predict the gender
if __name__ == '__main__':
    if platform.system() == 'Linux':
        os.system('"Praat.exe" --run extract_freq_info.praat')
    elif platform.system() == 'Windows':
        os.system('"./Praat" --run extract_freq_info.praat')
    else:
        os.system('"Praat.app/Contents/MacOS/Praat" --run extract_freq_info.praat')

    #import output file (mean,min,max freq)
    file = open('output.txt','r')
    values = file.readline()
    values = values.split(', ')

    #train model
    print("training and tuning svm")
    df = import_and_clean()
    tuned_svm = parameter_tuning_svm(df)

    #predict gender
    values = np.array(values, dtype=np.float64)
    values = values.reshape(1, -1)
    predictions = tuned_svm.predict(values)
    if predictions == 0:
        print("you are a female")
    else:
        print("you are a male")

training and tuning svm
18
you are a female
